In [3]:
import numpy as np #linear algebra
import pandas as pd #data processing

import os
import re
import nltk


In [5]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns

In [7]:
train_data=pd.read_csv('./train.csv')
test_data=pd.read_csv('./test.csv')

In [9]:
train_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
test_data.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [13]:
print(train_data.isnull().sum())
print('************')
print(test_data.isnull().sum())
 


id           0
title      558
author    1957
text        39
label        0
dtype: int64
************
id          0
title     122
author    503
text        7
dtype: int64


In [15]:
test_data=test_data.fillna(' ')
train_data=train_data.fillna(' ')
test_data['total']=test_data['title']+' '+test_data['author']+test_data['text']
train_data['total']=train_data['title']+' '+train_data['author']+train_data['text']

# Creating Workcloud Visuals

In [12]:
real_words = ''
fake_words = ''
stopwords = set(STOPWORDS) 
  
# iterate through the csv file 
for val in train_data[train_data['label']==1].text: 
  
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    real_words += " ".join(tokens)+" "

for val in train_data[train_data['label']==0].text: 
      
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    fake_words += " ".join(tokens)+" "

KeyboardInterrupt: 

In [ ]:
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(fake_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

In [ ]:
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(real_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

In [ ]:
# Drop rows with NaN values in the 'text' column
train_data = train_data.dropna(subset=['text'])
test_data = test_data.dropna(subset=['text'])

# Cleaning And PreProcessing

# 1 REGEX

In [17]:
#Remove punctuations from the String  
s = "!</> hello $$ </>^this is !!!my %f%i&rst&% $@@@project^^^&&!& </>*@# &&\ @@@##%^^&!@# %%$"
s = re.sub(r'[^\w\s]','',s)
print(s)

 hello  this is my first project    


In [19]:
def clean_text(text):
    # Remove non-alphanumeric characters and convert to lowercase
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    return cleaned_text

# 2 Tokenization

In [21]:
nltk.word_tokenize("Hello how are you")

['Hello', 'how', 'are', 'you']

# 3 StopWords

In [23]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [25]:
sentence = "Covid-19 pandemic has impacted many countries and what it did to economy is very stressful "
words = nltk.word_tokenize(sentence)
words = [w for w in words if w not in stop_words]
words

['Covid-19',
 'pandemic',
 'impacted',
 'many',
 'countries',
 'economy',
 'stressful']

# 4 Lemmatization


In [27]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [29]:
input_str="been had done languages cities mice"

# tokenize the sentence 
input_str=nltk.word_tokenize(input_str)

#Lemmatization each word
for word in input_str:
    print(lemmatizer.lemmatize(word))


been
had
done
language
city
mouse


# Apply

In [31]:
lemmatizer=WordNetLemmatizer()
for index,row in train_data.iterrows():
    filter_sentence = ''
    
    sentence = row['total']
    sentence = re.sub(r'[^\w\s]','',sentence) #cleaning
    
    words = nltk.word_tokenize(sentence) #tokenization
    
    words = [w for w in words if not w in stop_words]  #stopwords removal
    
    for word in words:
        filter_sentence = filter_sentence + ' ' + str(lemmatizer.lemmatize(word)).lower()
        
    train_data.loc[index,'total'] = filter_sentence


In [49]:
train = train_data[['total','label']]

# Applying NLP techniques


In [52]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/suhanibisht/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/suhanibisht/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [54]:
# Tokenization, lowercase, and remove stop words
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

In [56]:
train_data['processed_text'] = train_data['text'].apply(preprocess_text)
test_data['processed_text'] = test_data['text'].apply(preprocess_text)

# Bag of Words/ CountVectorizer

In [57]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out())

In [ ]:
print(X.toarray())

In [ ]:
# Feature extraction using count vectorization and tfidf.
count_vectorizer = CountVectorizer()
freq_term_matrix = count_vectorizer.fit_transform(train['total'])

# TF-IDF Vectorization

In [ ]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train = tfidf_vectorizer.fit_transform(train_data['processed_text'])
y_train = train_data['label']

X_test = tfidf_vectorizer.transform(test_data['processed_text'])

# Pickle

In [ ]:
from sklearn.linear_model import LogisticRegression
import pickle
import os

In [ ]:
# Create 'models' directory if it doesn't exist
models_dir = 'models'
os.makedirs(models_dir, exist_ok=True)

In [ ]:
# Create and train the Logistic Regression model
logreg_model = LogisticRegression()
logreg_model.fit(X_train, y_train)

In [ ]:
# Save the model and TF-IDF vectorizer
model_filepath = os.path.join(models_dir, 'logreg_model.pkl')
vectorizer_filepath = os.path.join(models_dir, 'tfidf_vectorizer.pkl')

In [ ]:
with open(model_filepath, 'wb') as model_file:
    pickle.dump(logreg_model, model_file)

with open(vectorizer_filepath, 'wb') as vectorizer_file:
    pickle.dump(tfidf_vectorizer, vectorizer_file)

In [66]:
print("numpy", np.__version__)
print("pandas", pd.__version__)
import sklearn
print("sklearn", sklearn.__version__)
print("nltk", nltk.__version__)

numpy 1.26.4
pandas 2.1.4
sklearn 1.2.2
nltk 3.8.1
